In [1]:
import transformers
import torch

#*** pretrained_model_vectorizer.py
# Library which includes all code that involves the use of pretrained
# models for contextualization
# Functions:
#    vectorize_with_pretrained_embeddings(sentences, labels): Given a list of text examples,
#    produces embeddings of dim 768 for each example
#***


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/'
f= pd.read_csv("data_passage_ques_options_pv.csv")

/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data


In [12]:
f.head()

,pdfname,PassNumUnq,Passage,QNoUnq,QuestionText,OptionAddInfo,option_correct_ans,option_distractor1,option_distractor2,option_distractor3,pVal,question_correct_ans,question_dis1,question_dis2,question_dis3,distractors,psg_question_correct_ans,psg_question_dis1,psg_question_dis2,psg_question_dis3
0,2019-released-items-ela-g4,2019-released-items-ela-g4.1,"""No one but a man can do this,"" the business ...",2019-released-items-ela-g4.1.13,"What does the phrase ""set out"" mean as it is ...",No,began her journey,grabbed her suitcase,accepted work,started writing,0.85,"What does the phrase ""set out"" mean as it is ...","What does the phrase ""set out"" mean as it is ...","What does the phrase ""set out"" mean as it is ...","What does the phrase ""set out"" mean as it is ...","What does the phrase ""set out"" mean as it is ...","began her journey What does the phrase ""set ou...","What does the phrase ""set out"" mean as it is ...","What does the phrase ""set out"" mean as it is ...","What does the phrase ""set out"" mean as it is ..."
1,2019-released-items-ela-g4,2019-released-items-ela-g4.1,"""No one but a man can do this,"" the business ...",2019-released-items-ela-g4.1.14,Read this sentence from paragraph 6.\n *Trave...,No,It shows one way to help her reach her goal.,It shows how she is the same as the character ...,It shows a young reporter exploring the world ...,It shows that a young woman taking a trip alon...,0.51,Read this sentence from paragraph 6.\n *Trave...,Read this sentence from paragraph 6.\n *Trave...,Read this sentence from paragraph 6.\n *Trave...,Read this sentence from paragraph 6.\n *Trave...,Read this sentence from paragraph 6.\n *Trave...,It shows one way to help her reach her goal. R...,Read this sentence from paragraph 6.\n *Trave...,Read this sentence from paragraph 6.\n *Trave...,Read this sentence from paragraph 6.\n *Trave...
2,2019-released-items-ela-g4,2019-released-items-ela-g4.1,"""No one but a man can do this,"" the business ...",2019-released-items-ela-g4.1.15,How does the author organize the information ...,No,by listing events in the order they happened,by comparing and contrasting the places Nellie...,by showing what caused Nellie to want to take ...,by stating how the problem of traveling so far...,0.61,How does the author organize the information ...,How does the author organize the information ...,How does the author organize the information ...,How does the author organize the information ...,How does the author organize the information ...,by listing events in the order they happened H...,How does the author organize the information ...,How does the author organize the information ...,How does the author organize the information ...
3,2019-released-items-ela-g4,2019-released-items-ela-g4.1,"""No one but a man can do this,"" the business ...",2019-released-items-ela-g4.1.16,"What does the word ""journalism"" mean as it is...",No,writing for newspapers,traveling for women,finding adventures,discovering opinions,0.62,"What does the word ""journalism"" mean as it is...","What does the word ""journalism"" mean as it is...","What does the word ""journalism"" mean as it is...","What does the word ""journalism"" mean as it is...","What does the word ""journalism"" mean as it is...","writing for newspapers What does the word ""jou...","What does the word ""journalism"" mean as it is...","What does the word ""journalism"" mean as it is...","What does the word ""journalism"" mean as it is..."
4,2019-released-items-ela-g4,2019-released-items-ela-g4.1,"""No one but a man can do this,"" the business ...",2019-released-items-ela-g4.1.17,Which detail from the article does the map su...,No,"""She met her challenge!"" (paragraph 8)","""Start the man, and I'll start the same day."" ...","""She reached New York in 41/2 days."" (paragrap...","""She reported to her readers what she saw, tho...",0.38,Which detail from the article does the map su...,Which detail from the article does the map su...,Which detail 

In [4]:
question= f['QuestionText']
option1= f['option_correct_ans']
option2= f['question_dis1']
option3= f['question_dis2']
option4= f['question_dis3']
ques_correct_answer= f['question_correct_ans']
ques_distractor1= f['option_distractor1']
ques_distractor2= f['option_distractor2']
ques_distractor3= f['option_distractor3']



KeyError: 'option_correct_ans'

In [13]:

def vectorize_with_pretrained_embeddings(sentences):
  """
  Produces a tensor containing a BERT embedding for each sentence in the dataset or in a
  batch
  Args:
    sentences: List of sentences of length n
  Returns:
    embeddings: A 2D torch array containing embeddings for each of the n sentences (n x d)
                where d = 768
  """

  tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased') #,padding=True, truncation=True
  pretrained_model = transformers.BertModel.from_pretrained('bert-base-cased', output_hidden_states=False)
  pretrained_model.eval()
  embeddings = []
  for sentence in sentences:
    with_tags = "[CLS] " + sentence + " [SEP]"
    tokenized_sentence = tokenizer.tokenize(with_tags)
    #print(tokenized_sentence)
    #print(len(tokenized_sentence))
    indices_from_tokens = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    segments_ids = [1] * len(indices_from_tokens)
    tokens_tensor = torch.tensor([indices_from_tokens])
    segments_tensors = torch.tensor([segments_ids])
    tokens_tensor = tokens_tensor[:,:512]
    segments_tensors = segments_tensors[:,:512]
    #print(indices_from_tokens)
    # print(tokens_tensor)
    # print(segments_tensors)
    with torch.no_grad():
      outputs = pretrained_model(tokens_tensor, segments_tensors)[0] # The output is the
      #last hidden state of the pretrained model of shape 1 x sentence_length x BERT embedding_length
      embeddings.append(torch.mean(outputs, dim = 1))# we average across the embedding length
      #dimension to produce constant sized tensors
  print(embeddings[0].shape)
  embeddings = torch.cat(embeddings, dim = 0)
  print('Shape of embeddings tensor (n x d = 768): ', embeddings.shape)
  return embeddings.cpu().detach().numpy()


In [ ]:
#Test sentences. To test, run 'python pretrained_model_vectorizer.py' at the command line
#sentences = ["The boy is running", "The dog has been barking for the whole evening"]
#To use with real data, import pretrained_model_vectorizer, then call
#'vectorize_with_pretrained_embeddings' on your list of sentences to embed
#question.embed= vectorize_with_pretrained_embeddings(question)

torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1116, 768])


In [58]:
quesoptionembed_corr= vectorize_with_pretrained_embeddings(ques_correct_answer)
quesoptionembed1= vectorize_with_pretrained_embeddings(ques_distractor1)
quesoptionembed2= vectorize_with_pretrained_embeddings(ques_distractor2)
quesoptionembed3= vectorize_with_pretrained_embeddings(ques_distractor3)

In [15]:
input= f['distractors']
quesalldistractors= vectorize_with_pretrained_embeddings(input)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])


In [16]:

%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/'
df= pd.DataFrame(quesalldistractors)
df.to_csv('question_embed_alldistractors.csv')

/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data


In [20]:
# counter=0
# out=[]
# for sentence in psg_ques_distractor1:
#   counter+=1
#   print(counter)
#   try:
#     psg_quesoptionembed_corr= vectorize_with_pretrained_embeddings(sentence)
#     out.append(psg_quesoptionembed_corr)
#   except RuntimeError as e:
#     print(e)
#     print(sentence)
#     continue


psg_quesoptionembed1= vectorize_with_pretrained_embeddings(f['psg_question_correct_ans'])
psg_quesoptionembed2= vectorize_with_pretrained_embeddings(f['psg_question_dis1'])
psg_quesoptionembed3= vectorize_with_pretrained_embeddings(f['psg_question_dis2'])
psg_quesoptionembed4= vectorize_with_pretrained_embeddings(f['psg_question_dis3'])

KeyboardInterrupt: 

In [1]:
out=[]
out[correct=psg_quesoptionembed1]
out[distract1= psg_quesoptionembed2]
out[distract2= psg_quesoptionembed3]
out[distract3= psg_quesoptionembed3]
df= pd.DataFrame(out)
df.to_csv('passage_question_option.csv')

NameError: name 'psg_quesoptionembed1' is not defined

In [14]:
quesembed= vectorize_with_pretrained_embeddings(question)
corrembed= vectorize_with_pretrained_embeddings(option1)
optionembed1= vectorize_with_pretrained_embeddings(option2)
optionembed2= vectorize_with_pretrained_embeddings(option3)
optionembed3= vectorize_with_pretrained_embeddings(option4)
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/'


torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])
torch.Size([1, 768])
Shape of embeddings tensor (n x d = 768):  torch.Size([1053, 768])


In [15]:
import numpy as np
from numpy import savetxt
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/'
df= pd.DataFrame(quesoptionembed_corr)

# df.to_csv('question_embed_correct.csv')

# df= pd.DataFrame(quesoptionembed1)
# df.to_csv('question_embed_dis1.csv')

# df= pd.DataFrame(quesoptionembed2)
# df.to_csv('question_embed_dis2.csv')

# df= pd.DataFrame(quesoptionembed3)
# df.to_csv('question_embed_dis3.csv')


# df= pd.DataFrame(quesembed)
# df.to_csv('question_embed.csv')

# df= pd.DataFrame(corrembed)
# df.to_csv('option_embed_correct.csv')

# df= pd.DataFrame(optionembed1)
# df.to_csv('optionembed1.csv')

# df= pd.DataFrame(optionembed2)
# df.to_csv('optionembed2.csv')

# df= pd.DataFrame(optionembed3)
# df.to_csv('optionembed3.csv')

#np.savetxt('/Users/radhika/Downloads/question_embed.csv', question.embed, delimiter=',')
#np.save(question.embed,)


Similarity measures between embeddings

In [112]:
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/'
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
##  difference between distractors only



##  difference between question + distractors
quesoptionembed_corr= pd.read_csv('question_embed_correct.csv')
quesoptionembed1= pd.read_csv('question_embed_dis1.csv')
quesoptionembed2= pd.read_csv('question_embed_dis2.csv')
quesoptionembed3= pd.read_csv('question_embed_dis3.csv')


## drop the index column
quesoptionembed_corr = quesoptionembed_corr.iloc[: , 1:]
quesoptionembed1= quesoptionembed1.iloc[: , 1:]
quesoptionembed2= quesoptionembed2.iloc[: , 1:]
quesoptionembed3= quesoptionembed3.iloc[: , 1:]

## convert to torch
quesoptionembed_corr=torch.Tensor(quesoptionembed_corr.to_numpy())
quesoptionembed1=torch.Tensor(quesoptionembed1.to_numpy())
quesoptionembed2=torch.Tensor(quesoptionembed2.to_numpy())
quesoptionembed3=torch.Tensor(quesoptionembed3.to_numpy())

## check for one row
# x=quesoptionembed1[767,:]
# y=quesoptionembed_corr[767,:]
# x=torch.Tensor(x).reshape(1,-1)
# y=torch.Tensor(y).reshape(1,-1)
# print(torch.cosine_similarity(x, y))

## Find distance
sim_corr_dis1=torch.cosine_similarity(quesoptionembed_corr, quesoptionembed1)
sim_corr_dis2=torch.cosine_similarity(quesoptionembed_corr, quesoptionembed2)
sim_corr_dis3=torch.cosine_similarity(quesoptionembed_corr, quesoptionembed3)
out=torch.column_stack((sim_corr_dis1, sim_corr_dis2,sim_corr_dis3))
print(out.shape)
pd.DataFrame(out).to_csv('sim_corr_distractors.csv', index=False)



/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data


In [5]:
%cd '/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data/'
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
##  difference between distractors only

##  difference between question + distractors
quesoptionembed_corr= pd.read_csv('option_embed_correct.csv')
quesoptionembed1= pd.read_csv('optionembed1.csv')
quesoptionembed2= pd.read_csv('optionembed2.csv')
quesoptionembed3= pd.read_csv('optionembed3.csv')


## drop the index column
quesoptionembed_corr = quesoptionembed_corr.iloc[: , 1:]
quesoptionembed1= quesoptionembed1.iloc[: , 1:]
quesoptionembed2= quesoptionembed2.iloc[: , 1:]
quesoptionembed3= quesoptionembed3.iloc[: , 1:]

## convert to torch
quesoptionembed_corr=torch.Tensor(quesoptionembed_corr.to_numpy())
quesoptionembed1=torch.Tensor(quesoptionembed1.to_numpy())
quesoptionembed2=torch.Tensor(quesoptionembed2.to_numpy())
quesoptionembed3=torch.Tensor(quesoptionembed3.to_numpy())

## check for one row
# x=quesoptionembed1[767,:]
# y=quesoptionembed_corr[767,:]
# x=torch.Tensor(x).reshape(1,-1)
# y=torch.Tensor(y).reshape(1,-1)
# print(torch.cosine_similarity(x, y))

## Find distance
sim_corr_dis1=torch.cosine_similarity(quesoptionembed_corr, quesoptionembed1)
sim_corr_dis2=torch.cosine_similarity(quesoptionembed_corr, quesoptionembed2)
sim_corr_dis3=torch.cosine_similarity(quesoptionembed_corr, quesoptionembed3)
out=torch.column_stack((sim_corr_dis1, sim_corr_dis2,sim_corr_dis3))
print(out.shape)
pd.DataFrame(out).to_csv('sim_corr_distractors_only.csv', index=False)



/content/drive/MyDrive/0. Projects - Stanford/Item generation/Data
torch.Size([1053, 3])


(1053, 768)

tensor([0.8070])
